# H→γγ MVA Training Notebook

This notebook trains a Neural Network to distinguish H→γγ signal events from background events using Monte Carlo (MC) simulations.
The trained model will then be used in the main H→γγ analysis notebook (`HyyAnalysisNew.ipynb`) to improve event selection.

Steps:
1. Configure paths to MC signal and background samples.
2. Define features for the MVA.
3. Load and preprocess MC data.
4. Train a Neural Network classifier.
5. Evaluate the model's performance.
6. Save the trained model and feature scaler.

## 1. Imports

In [11]:
import uproot
import pandas as pd
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import os
import glob
import joblib # For saving/loading model and scaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix
import vector # For 4-vector calculations

# Plotting style
plt.style.use('seaborn-v0_8-pastel')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Ensure the vector library is aware of 'mass' attribute if not already default
# vector.register_awkward() # May not be needed depending on vector version

## 2. Configuration

In [12]:
# --- User Configuration ---

# Base directory (assuming this notebook is in 'higgs_lovers')
notebook_base_dir = os.getcwd()
data_dir = os.path.join(notebook_base_dir, "data") # Standard data directory

# !! PLEASE UPDATE THESE PATHS TO YOUR H→γγ MC FILES !!
# Example placeholder paths:
mc_signal_hyy_file_pattern = os.path.join(data_dir, "DAOD_PHYSLITE.38191712.*.pool.root.1") # e.g., ggH production
mc_background_hyy_file_pattern = os.path.join(data_dir, "DAOD_PHYSLITE.37110937._000011.pool.root.1") # e.g., direct diphoton production
# You might have multiple background sources to combine.

# Tree name in your MC ROOT files
tree_name = "analysis" # Same as in HyyAnalysisNew.ipynb

# Output paths for the trained model and scaler
output_model_dir = notebook_base_dir
mva_model_path = os.path.join(output_model_dir, "hyy_mva_model.pkl")
mva_scaler_path = os.path.join(output_model_dir, "hyy_mva_scaler.pkl")

# Branches to read from MC files (ensure these cover all features needed for MVA and pre-selection)
# These should be similar to HyyAnalysisNew.ipynb, plus any additional ones for MVA features.
branches_to_read_mc = [
    "photon_pt", "photon_eta", "photon_phi", "photon_e",
    "photon_isTightID", "photon_ptcone20",
    "mcEventWeight" # Assuming there's an MC event weight branch
    # Add other branches if needed for more sophisticated features
]

# Define MVA features (list of column names that will be created in the DataFrame)
# IMPORTANT: Do NOT use m_yy (diphoton invariant mass) directly if you plan to fit the m_yy spectrum later.
mva_features_list = [
    'ph1_pt_norm', 'ph2_pt_norm', # Normalized pT by m_yy (can be okay if m_yy itself isn't a feature)
    'ph1_eta', 'ph2_eta',
    'delta_eta_yy', 'delta_phi_yy', 'delta_R_yy',
    'ph1_ptcone20_norm', 'ph2_ptcone20_norm', # Isolation normalized by photon pT
    # 'ph1_isTightID', 'ph2_isTightID', # Can be used if converted to numerical (0 or 1)
    'cos_theta_cs_yy' # Cosine of Collins-Soper angle (sensitive to spin)
    # Add more features as desired
]


print("Configuration set.")
print(f"MC Signal pattern: {mc_signal_hyy_file_pattern}")
print(f"MC Background pattern: {mc_background_hyy_file_pattern}")
print(f"MVA Model will be saved to: {mva_model_path}")
print(f"MVA Scaler will be saved to: {mva_scaler_path}")

Configuration set.
MC Signal pattern: /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712.*.pool.root.1
MC Background pattern: /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.37110937._000011.pool.root.1
MVA Model will be saved to: /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/hyy_mva_model.pkl
MVA Scaler will be saved to: /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/hyy_mva_scaler.pkl


## 3. Data Loading and Preprocessing Function for MC

This function will load data from ROOT files, apply initial selections (similar to `HyyAnalysisNew.ipynb` but adapted for MC and MVA features), and calculate MVA input variables.

In [13]:
def load_and_preprocess_mc_for_mva(file_paths_pattern, is_signal_flag, tree_name_param, branches_param):
    """
    Loads and preprocesses MC data for MVA training.
    - Reads specified branches.
    - Applies basic quality cuts (from HyyAnalysisNew.ipynb).
    - Calculates MVA features.
    - Returns a pandas DataFrame.
    """
    all_files = glob.glob(file_paths_pattern)
    if not all_files:
        print(f"No files found for pattern: {file_paths_pattern}")
        return pd.DataFrame()

    print(f"Processing {len(all_files)} files for pattern: {file_paths_pattern}...")
    
    df_list = []

    for file_path in all_files:
        try:
            with uproot.open(file_path + ":" + tree_name_param) as tree:
                # Limit entries for faster testing if needed: entry_stop="10000"
                events = tree.arrays(branches_param, library="ak") 
        except Exception as e:
            print(f"Error opening or reading file {file_path}: {e}")
            continue

        if len(events) == 0:
            print(f"No events in {file_path}")
            continue

        # --- Apply Pre-selections (adapted from HyyAnalysisNew.ipynb) ---
        # Ensure at least 2 photons
        n_photons = ak.num(events.photon_pt)
        events = events[n_photons >= 2]
        if len(events) == 0: continue

        # 1. Photon Reconstruction Quality (isTightID)
        # Consider only events where the two leading photons pass isTightID
        # (HyyAnalysisNew applies this after selecting 2 photons)
        # For MVA, we might want to keep events even if one is not tight and let MVA learn
        # For simplicity here, let's require both leading to be tight initially.
        # This can be relaxed, and isTightID can become an MVA feature.
        tight_id_cut = (events.photon_isTightID[:,0] == True) & (events.photon_isTightID[:,1] == True)
        events = events[tight_id_cut]
        if len(events) == 0: continue
            
        # 2. Photon pT cuts (leading > 50 GeV, sub-leading > 30 GeV)
        # These are relatively high cuts from the paper. For MVA, sometimes looser cuts are used.
        # Let's keep them for now as a baseline.
        pt_cut = (events.photon_pt[:,0] > 50) & (events.photon_pt[:,1] > 30) # Assuming GeV
        events = events[pt_cut]
        if len(events) == 0: continue

        # 3. Calorimeter Isolation (ptcone20 / pt < 0.055)
        iso_cut = ((events.photon_ptcone20[:,0] / events.photon_pt[:,0]) < 0.055) & \
                    ((events.photon_ptcone20[:,1] / events.photon_pt[:,1]) < 0.055)
        events = events[iso_cut]
        if len(events) == 0: continue

        # 4. Eta Transition Region (|eta| not between 1.37 and 1.52)
        eta_cut_ph0 = ~((np.abs(events.photon_eta[:,0]) > 1.37) & (np.abs(events.photon_eta[:,0]) < 1.52))
        eta_cut_ph1 = ~((np.abs(events.photon_eta[:,1]) > 1.37) & (np.abs(events.photon_eta[:,1]) < 1.52))
        events = events[eta_cut_ph0 & eta_cut_ph1]
        if len(events) == 0: continue

        # --- Calculate Diphoton Invariant Mass (m_yy) ---
        # Needed for some normalized features, but NOT as a direct MVA input.
        p4_photons = vector.zip({
            "pt": events.photon_pt, "eta": events.photon_eta,
            "phi": events.photon_phi, "e": events.photon_e
        })
        diphoton = p4_photons[:,0] + p4_photons[:,1]
        m_yy = diphoton.mass
        
        # 5. Mass-based Isolation (pt_gamma / m_yy > 0.35)
        # This cut from HyyAnalysisNew uses m_yy.
        mass_iso_cut = (events.photon_pt[:,0] / m_yy > 0.35) & (events.photon_pt[:,1] / m_yy > 0.35)
        events = events[mass_iso_cut]
        if len(events) == 0: continue
        
        # Re-calculate m_yy for the events that passed all cuts
        p4_photons_final = vector.zip({
            "pt": events.photon_pt, "eta": events.photon_eta,
            "phi": events.photon_phi, "e": events.photon_e
        })
        diphoton_final = p4_photons_final[:,0] + p4_photons_final[:,1]
        m_yy_final = diphoton_final.mass
        pt_yy_final = diphoton_final.pt # Transverse momentum of the diphoton system

        # --- Feature Engineering for MVA ---
        # Ensure we only use the leading two photons for these features
        ph1_pt = events.photon_pt[:,0]
        ph2_pt = events.photon_pt[:,1]
        ph1_eta = events.photon_eta[:,0]
        ph2_eta = events.photon_eta[:,1]
        ph1_phi = events.photon_phi[:,0]
        ph2_phi = events.photon_phi[:,1]
        ph1_e = events.photon_e[:,0] # Not directly used in example features but good to have
        ph2_e = events.photon_e[:,1] # Not directly used in example features but good to have

        ph1_ptcone20 = events.photon_ptcone20[:,0]
        ph2_ptcone20 = events.photon_ptcone20[:,1]
        
        # Normalized pTs (by m_yy)
        ph1_pt_norm = ph1_pt / m_yy_final
        ph2_pt_norm = ph2_pt / m_yy_final
        
        # Angular variables
        delta_eta_yy = np.abs(ph1_eta - ph2_eta)
        
        # Delta phi (handle wrap-around)
        delta_phi_yy_raw = ph1_phi - ph2_phi
        delta_phi_yy = np.abs(np.mod(delta_phi_yy_raw + np.pi, 2 * np.pi) - np.pi)

        delta_R_yy = np.sqrt(delta_eta_yy**2 + delta_phi_yy**2)
        
        # Normalized isolation
        ph1_ptcone20_norm = ph1_ptcone20 / ph1_pt
        ph2_ptcone20_norm = ph2_ptcone20 / ph2_pt
        
        # Collins-Soper frame cosine theta star
        # P_yy = ph1_p4 + ph2_p4 (diphoton system 4-momentum)
        # In lab frame: p_z_yy = ph1_pz + ph2_pz; E_yy = ph1_E + ph2_E
        # Boost to CS frame (where P_yy has only Pz_cs component) is complex.
        # A simpler version for H->gg:
        cos_theta_cs_yy = np.abs(np.tanh(0.5 * (ph1_eta - ph2_eta))) # Approximation often used

        # Event weights
        event_weights = events.mcEventWeight if "mcEventWeight" in events.fields else ak.ones_like(ph1_pt)
        # If mcEventWeight is an array per event, take the first one (nominal)
        if hasattr(event_weights, "ndim") and event_weights.ndim > 1 and event_weights.type.content.length > 0 :
             event_weights = event_weights[:,0]


        # Create a dictionary for the DataFrame
        df_dict = {
            'ph1_pt_norm': ak.to_numpy(ph1_pt_norm),
            'ph2_pt_norm': ak.to_numpy(ph2_pt_norm),
            'ph1_eta': ak.to_numpy(ph1_eta),
            'ph2_eta': ak.to_numpy(ph2_eta),
            'delta_eta_yy': ak.to_numpy(delta_eta_yy),
            'delta_phi_yy': ak.to_numpy(delta_phi_yy),
            'delta_R_yy': ak.to_numpy(delta_R_yy),
            'ph1_ptcone20_norm': ak.to_numpy(ph1_ptcone20_norm),
            'ph2_ptcone20_norm': ak.to_numpy(ph2_ptcone20_norm),
            'cos_theta_cs_yy': ak.to_numpy(cos_theta_cs_yy),
            # 'ph1_isTightID': ak.to_numpy(events.photon_isTightID[:,0]).astype(int), # Example if used
            # 'ph2_isTightID': ak.to_numpy(events.photon_isTightID[:,1]).astype(int), # Example if used
            'm_yy': ak.to_numpy(m_yy_final), # Keep for potential plotting/checks, but NOT an MVA input feature
            'pt_yy': ak.to_numpy(pt_yy_final), # Keep for potential plotting/checks
            'eventWeight': ak.to_numpy(event_weights),
            'isSignal': is_signal_flag
        }
        df_list.append(pd.DataFrame(df_dict))

    if not df_list:
        return pd.DataFrame()
        
    final_df = pd.concat(df_list, ignore_index=True)
    final_df.dropna(inplace=True) # Clean up any NaNs that might have slipped through
    print(f"Finished processing for pattern. Resulting DataFrame shape: {final_df.shape}")
    return final_df

print("MC loading and preprocessing function defined.")

MC loading and preprocessing function defined.


## 4. Load and Process MC Signal and Background Data

In [14]:
# Load Signal MC
df_signal_mc = load_and_preprocess_mc_for_mva(mc_signal_hyy_file_pattern, 1, tree_name, branches_to_read_mc)
print(f"Loaded Signal MC. Shape: {df_signal_mc.shape}")
if not df_signal_mc.empty:
    print("Signal MC columns:", df_signal_mc.columns)
    print("Signal MC head:\n", df_signal_mc.head())

# Load Background MC
df_background_mc = load_and_preprocess_mc_for_mva(mc_background_hyy_file_pattern, 0, tree_name, branches_to_read_mc)
print(f"Loaded Background MC. Shape: {df_background_mc.shape}")
if not df_background_mc.empty:
    print("Background MC columns:", df_background_mc.columns)
    print("Background MC head:\n", df_background_mc.head())

# Combine signal and background
if not df_signal_mc.empty and not df_background_mc.empty:
    df_combined_mc = pd.concat([df_signal_mc, df_background_mc], ignore_index=True)
    print(f"Combined MC DataFrame shape: {df_combined_mc.shape}")
    # Shuffle the DataFrame
    df_combined_mc = df_combined_mc.sample(frac=1, random_state=42).reset_index(drop=True)
    print("Combined MC DataFrame head after shuffle:\n", df_combined_mc.head())
elif not df_signal_mc.empty:
    print("Warning: Background MC is empty. Using only signal MC.")
    df_combined_mc = df_signal_mc.copy()
elif not df_background_mc.empty:
    print("Warning: Signal MC is empty. Using only background MC.")
    df_combined_mc = df_background_mc.copy()
else:
    print("Error: Both signal and background MC DataFrames are empty. Cannot proceed.")
    df_combined_mc = pd.DataFrame() # Ensure it's defined

Processing 17 files for pattern: /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712.*.pool.root.1...
Error opening or reading file /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712._000016.pool.root.1: [Errno 2] No such file or directory: '/Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712._000016.pool.root.1:analysis'
Error opening or reading file /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712._000006.pool.root.1: [Errno 2] No such file or directory: '/Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712._000006.pool.root.1:analysis'
Error opening or reading file /Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712._000013.pool.root.1: [Errno 2] No such file or directory: '/Users/xin/Documents/Documents/College/Phys_392/higgs_lovers/data/DAOD_PHYSLITE.38191712.

### Quick Check of Feature Distributions (Optional)

In [15]:
if not df_combined_mc.empty:
    for feature in mva_features_list:
        if feature in df_combined_mc.columns:
            plt.figure(figsize=(8, 5))
            plt.hist(df_combined_mc[df_combined_mc['isSignal']==1][feature], bins=50, label='Signal', alpha=0.7, density=True, weights=df_combined_mc[df_combined_mc['isSignal']==1]['eventWeight'])
            plt.hist(df_combined_mc[df_combined_mc['isSignal']==0][feature], bins=50, label='Background', alpha=0.7, density=True, weights=df_combined_mc[df_combined_mc['isSignal']==0]['eventWeight'])
            plt.title(f'Distribution of {feature}')
            plt.xlabel(feature)
            plt.ylabel('Normalized Events')
            plt.legend()
            plt.yscale('log')
            plt.show()
        else:
            print(f"Warning: Feature '{feature}' not found in DataFrame for plotting.")
else:
    print("Skipping feature plots as combined MC DataFrame is empty.")

Skipping feature plots as combined MC DataFrame is empty.


## 5. Define Features (X) and Target (y)

In [16]:
if not df_combined_mc.empty:
    # Check if all mva_features_list are in df_combined_mc.columns
    missing_features = [f for f in mva_features_list if f not in df_combined_mc.columns]
    if missing_features:
        print(f"Error: The following MVA features are missing from the DataFrame: {missing_features}")
        print(f"Available columns: {df_combined_mc.columns.tolist()}")
        # Handle error appropriately, e.g., by raising an exception or exiting
        X = pd.DataFrame() # Empty dataframe
        y = pd.Series()
        weights = pd.Series()
    else:
        X = df_combined_mc[mva_features_list]
        y = df_combined_mc['isSignal']
        weights = df_combined_mc['eventWeight'] # Sample weights for training
        print("Features (X) and Target (y) defined.")
        print("X shape:", X.shape)
        print("y shape:", y.shape)
        print("weights shape:", weights.shape)
else:
    print("Cannot define X and y as combined MC DataFrame is empty.")
    X = pd.DataFrame()
    y = pd.Series()
    weights = pd.Series()

Cannot define X and y as combined MC DataFrame is empty.


## 6. Split Data and Scale Features

In [17]:
if not X.empty:
    # Split data into training and testing sets
    # Using stratify to ensure similar class proportions in train and test sets
    X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
        X, y, weights, test_size=0.3, random_state=42, stratify=y if y.nunique() > 1 else None
    )

    print(f"Training set size: {X_train.shape[0]}")
    print(f"Test set size: {X_test.shape[0]}")

    # Scale features
    # Important: Fit scaler ONLY on training data, then transform both train and test
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("Features scaled.")
else:
    print("Skipping data splitting and scaling as X is empty.")
    X_train_scaled = np.array([])
    X_test_scaled = np.array([])
    y_train = pd.Series()
    y_test = pd.Series()
    w_train = pd.Series()
    w_test = pd.Series()

Skipping data splitting and scaling as X is empty.


## 7. Train Neural Network
Using MLPClassifier from scikit-learn.

In [18]:
if X_train_scaled.size > 0 :
    # Define the MLP model
    # These parameters can be tuned (e.g., using GridSearchCV)
    mlp_model = MLPClassifier(hidden_layer_sizes=(64, 32), # Example: 2 hidden layers
                              activation='relu',
                              solver='adam',
                              alpha=0.0001, # L2 penalty (regularization)
                              batch_size='auto',
                              learning_rate='constant',
                              learning_rate_init=0.001,
                              max_iter=200, # Number of epochs
                              shuffle=True,
                              random_state=42,
                              early_stopping=True, # Stop training if validation score doesn't improve
                              n_iter_no_change=10, # Number of iterations with no improvement to wait
                              verbose=True)

    print("Training MLPClassifier model...")
    # Pass sample_weight to the fit method
    mlp_model.fit(X_train_scaled, y_train, # sample_weight=w_train # Not directly supported by MLPClassifier.fit for early stopping validation
                  # For weighted training with early stopping, one might need a custom loop or use a library
                  # that supports it more directly (like Keras/TensorFlow).
                  # For now, training unweighted for simplicity of early stopping, or remove early stopping if weights are crucial here.
                  # If weights are crucial, consider: mlp_model.fit(X_train_scaled, y_train) and evaluate with weights.
                 ) 
    # Re-fitting without early stopping if weights are essential for the main training loss
    # mlp_model = MLPClassifier(...) # re-init without early_stopping
    # mlp_model.fit(X_train_scaled, y_train, sample_weight=w_train) # This is how weights would be passed if not using early_stopping's validation set

    print("Model training complete.")

    # Evaluate on the test set
    accuracy = mlp_model.score(X_test_scaled, y_test) #, sample_weight=w_test) # score also takes sample_weight
    print(f"Model Accuracy on Test Set: {accuracy:.4f}")
else:
    print("Skipping model training as training data is empty.")
    mlp_model = None

Skipping model training as training data is empty.


## 8. Evaluate Model Performance

In [19]:
if mlp_model and X_test_scaled.size > 0:
    # Get predictions (probabilities for the positive class)
    y_pred_proba = mlp_model.predict_proba(X_test_scaled)[:, 1]
    y_pred_class = mlp_model.predict(X_test_scaled)

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba, sample_weight=w_test)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) - Test Set')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_class, sample_weight=w_test) # Use w_test for weighted CM
    print("Confusion Matrix (Test Set):\n", cm)
    
    # Plot MVA output score
    plt.figure()
    plt.hist(y_pred_proba[y_test==1], bins=50, weights=w_test[y_test==1], label='Signal (Test)', alpha=0.7, density=True)
    plt.hist(y_pred_proba[y_test==0], bins=50, weights=w_test[y_test==0], label='Background (Test)', alpha=0.7, density=True)
    plt.title('MVA Output Score Distribution (Test Set)')
    plt.xlabel('MVA Score')
    plt.ylabel('Normalized Events')
    plt.legend()
    plt.yscale('log')
    plt.show()

else:
    print("Skipping model evaluation as model or test data is not available.")

Skipping model evaluation as model or test data is not available.


## 9. Save Model and Scaler

In [20]:
if mlp_model and scaler:
    # Save the trained model
    joblib.dump(mlp_model, mva_model_path)
    print(f"Trained MVA model saved to: {mva_model_path}")

    # Save the scaler
    joblib.dump(scaler, mva_scaler_path)
    print(f"Scaler saved to: {mva_scaler_path}")
else:
    print("Model or scaler not available. Skipping saving.")

Model or scaler not available. Skipping saving.


## Next Steps:

1.  **Update MC File Paths**: Ensure `mc_signal_hyy_file_pattern` and `mc_background_hyy_file_pattern` in Cell 5 point to your actual H→γγ signal and background MC files. You might need to adjust the `branches_to_read_mc` and the feature engineering in `load_and_preprocess_mc_for_mva` (Cell 7) based on your MC content.
2.  **Run This Notebook**: Execute all cells to train the MVA and save the `hyy_mva_model.pkl` and `hyy_mva_scaler.pkl` files.
3.  **Modify `HyyAnalysisNew.ipynb`**: Proceed to integrate this trained MVA into your main analysis notebook to select events based on the MVA score.